In [1]:
# check architecture

if False:
    from keras.applications.vgg16 import VGG16

    vgg16 = VGG16(include_top=True, weights='imagenet', input_tensor=None, input_shape=None)
    vgg16.summary()

In [12]:
import h5py

path = '../../.keras/models/vgg16_weights_tf_dim_ordering_tf_kernels.h5'

f = h5py.File(path, 'r')
layer_names = list(f.keys())
corr_layers = dict.fromkeys(layer_names)
layer_names

['block1_conv1',
 'block1_conv2',
 'block1_pool',
 'block2_conv1',
 'block2_conv2',
 'block2_pool',
 'block3_conv1',
 'block3_conv2',
 'block3_conv3',
 'block3_pool',
 'block4_conv1',
 'block4_conv2',
 'block4_conv3',
 'block4_pool',
 'block5_conv1',
 'block5_conv2',
 'block5_conv3',
 'block5_pool',
 'fc1',
 'fc2',
 'flatten',
 'predictions']

In [7]:
from keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, GlobalAveragePooling2D
from keras.models import Model

x = h = Input((224,224,3))
h = Conv2D(64,(3,3),activation='relu',padding='same', name='c1_1')(h)
h = Conv2D(64,(3,3),activation='relu',padding='same', name='c1_2')(h)
#h = MaxPooling2D()(h)
#h = Conv2D(128,(3,3),activation='relu',padding='same', name='c2_1')(h)
#h = Conv2D(128,(3,3),activation='relu',padding='same', name='c2_2')(h)
#h = MaxPooling2D()(h)
#h = Conv2D(256,(3,3),activation='relu',padding='same', name='c3_1')(h)
#h = Conv2D(256,(3,3),activation='relu',padding='same', name='c3_2')(h)
#h = Conv2D(256,(3,3),activation='relu',padding='same', name='c3_3')(h)
#h = MaxPooling2D()(h)
#h = Conv2D(512,(3,3),activation='relu',padding='same', name='c4_1')(h)
#h = Conv2D(512,(3,3),activation='relu',padding='same', name='c4_2')(h)
#h = Conv2D(512,(3,3),activation='relu',padding='same', name='c4_3')(h)
#h = MaxPooling2D()(h)
#h = Conv2D(512,(3,3),activation='relu',padding='same', name='c5_1')(h)
#h = Conv2D(512,(3,3),activation='relu',padding='same', name='c5_2')(h)
#h = Conv2D(512,(3,3),activation='relu',padding='same', name='c5_3')(h)
#h = MaxPooling2D()(h)
#h = Flatten()(h)
#h = Dense(4096,activation='relu', name='fc_1')(h)
#h = Dense(4096,activation='relu', name='fc_2')(h)
#h = Dense(1000,activation='softmax', name='fc_3')(h)

h = GlobalAveragePooling2D()(h)
h = Dense(1)(h)
model = Model(x,h)

In [8]:
# layer mapping

corr_layers['block1_conv1'] = 'c1_1'
corr_layers['block1_conv2'] = 'c1_2'
corr_layers['block2_conv1'] = 'c2_1'
corr_layers['block2_conv2'] = 'c2_2'
corr_layers['block3_conv1'] = 'c3_1'
corr_layers['block3_conv2'] = 'c3_2'
corr_layers['block3_conv3'] = 'c3_3'
corr_layers['block4_conv1'] = 'c4_1'
corr_layers['block4_conv2'] = 'c4_2'
corr_layers['block4_conv3'] = 'c4_3'
corr_layers['block5_conv1'] = 'c5_1'
corr_layers['block5_conv2'] = 'c5_2'
corr_layers['block5_conv3'] = 'c5_3'

In [9]:
for pretrained_layer_name, model_layer_name in corr_layers.items():
    for layer in model.layers:
        if layer.get_config()['name']==model_layer_name:
            print(pretrained_layer_name + ' -> ' + model_layer_name)
#            print(list(f[pretrained_layer_name].keys()))
            w = f[pretrained_layer_name][pretrained_layer_name+'_W_1:0']
            b = f[pretrained_layer_name][pretrained_layer_name+'_b_1:0']
            layer.set_weights([w,b])

block1_conv1 -> c1_1
block1_conv2 -> c1_2


In [10]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
c1_1 (Conv2D)                (None, 224, 224, 64)      1792      
_________________________________________________________________
c1_2 (Conv2D)                (None, 224, 224, 64)      36928     
_________________________________________________________________
global_average_pooling2d_2 ( (None, 64)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 65        
Total params: 38,785
Trainable params: 38,785
Non-trainable params: 0
_________________________________________________________________
